In [2]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [3]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [4]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [5]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [6]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [7]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [8]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df


In [9]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권_상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [10]:
import pandas as pd

df = load_to_df('상권-추정유동인구')
df1 = df.copy()
# print(df1.head())
df1[df1['기준_년_분기_코드']=='2019_1']
df1[df1['기준_년_분기_코드']=='2019_2']


# df1.filter(like='여성').filter(like='토요일')
df1['여성_토요일_유동인구수'] = df1.filter(like='여성').filter(like='토요일').sum(axis=1)
df1['여성_토요일_유동인구수']
del_columns = df1.filter(like='여성').filter(like='토요일').filter(like='시간대').columns
df1.drop(del_columns, axis=1, inplace=True)
df1.filter(like='여성').filter(like='토요일')

delete_dict = {'여성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],
               '남성' : ['일요일','월요일','화요일','수요일','목요일','금요일','토요일'],
              }
delete_word = '시간대'




# len(df1.columns), len(df1.filter(like='유동인구_수').columns)
# df1.iloc[:,:5].head(2), df1.filter(like='유동인구_수').iloc[:,:5].head(2)
# df1['여성_토요일_유동인구수']
# df1.filter(items=['여성', '토요일'])

In [11]:
def delete_age_by_time(df1, delete_dict, delete_word):
    # df1.filter(like='여성').filter(like='토요일')
    for key, values in delete_dict.items():
        for value in values:
            df1[key+'_'+value+'_유동인구수'] = df1.filter(like=key).filter(like=value).sum(axis=1)
            del_columns = df1.filter(like=key).filter(like=value).filter(like=delete_word).columns
            df1.drop(del_columns, axis=1, inplace=True)
    
    return df1
    

In [12]:
df1 = delete_age_by_time(df1, delete_dict, delete_word)
df1.columns

Index(['기준_년_분기_코드', '상권_코드', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수',
       '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수',
       '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수', '시간대_1_유동인구_수', '시간대_2_유동인구_수',
       '시간대_3_유동인구_수', '시간대_4_유동인구_수', '시간대_5_유동인구_수', '시간대_6_유동인구_수',
       '월요일_유동인구_수', '화요일_유동인구_수', '수요일_유동인구_수', '목요일_유동인구_수', '금요일_유동인구_수',
       '토요일_유동인구_수', '일요일_유동인구_수', '여성_토요일_유동인구수', '여성_일요일_유동인구수',
       '여성_월요일_유동인구수', '여성_화요일_유동인구수', '여성_수요일_유동인구수', '여성_목요일_유동인구수',
       '여성_금요일_유동인구수', '남성_일요일_유동인구수', '남성_월요일_유동인구수', '남성_화요일_유동인구수',
       '남성_수요일_유동인구수', '남성_목요일_유동인구수', '남성_금요일_유동인구수', '남성_토요일_유동인구수'],
      dtype='object')

In [13]:
df1['총_시간_유동인구_수'] = df1.filter(like='시간대_').filter(like='_유동인구_수').sum(axis=1)
df1['총_연령_유동인구_수'] = df1.filter(like='연령대_').filter(like='0_유동인구_수').sum(axis=1)

del_columns = df1.filter(like='시간대_').filter(like='_유동인구_수').columns
# df1.drop(del_columns, axis=1, inplace=True)

del_columns = df1.filter(like='연령대_').filter(like='_유동인구_수').columns
# df1.drop(del_columns, axis=1, inplace=True)
df1['총_성별_유동인구_수'] = df1.filter(like='성_유').sum(axis=1)

In [14]:
df1.filter(like='총_')

,총_유동인구_수,총_시간_유동인구_수,총_연령_유동인구_수,총_성별_유동인구_수
0,118266,118267,102419,118265
1,246619,246621,190573,246619
2,144480,144479,123544,144480
3,166390,166387,149223,166390
4,65518,65518,58105,65517
...,...,...,...,...
2985,109383,109381,62764,109382
2986,112258,112258,87997,112259
2987,134058,134058,103687,134059
2988,39190,39190,31740,39190


## KNN 분류

In [15]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [16]:
def classify_knn (any_df, rs):  # (df, random_state)
    df = load_to_df(any_df)
    df_c = df.copy()
    df_c['평균_폐업_비율'] = df_c['폐업_영업_개월_평균'] / df_c['서울_폐업_영업_개월_평균']
    df_c['평균_운영_비율'] = df_c['운영_영업_개월_평균'] / df_c['서울_운영_영업_개월_평균']
    X, y = df_c[['평균_폐업_비율', '평균_운영_비율']], df_c['상권_변화_지표']
    X_train, X_test, y_train, y_test = \
       train_test_split(X, y, stratify=y, random_state=rs)
    train_accuracy = []
    test_accuracy = []

    for n in range(1, 11) :
        knn = KNeighborsClassifier(n_neighbors=n)
        knn.fit(X_train, y_train)
        train_accuracy.append(knn.score(X_train, y_train))
        test_accuracy.append(knn.score(X_test, y_test))
    
    return train_accuracy, test_accuracy
    
    

In [17]:
classify_knn('상권-상권변화지표', 3)

([0.9763992869875223,
  0.973475935828877,
  0.9762566844919786,
  0.9735828877005348,
  0.9741176470588235,
  0.9727629233511587,
  0.9723351158645276,
  0.9714438502673797,
  0.9711229946524064,
  0.969982174688057],
 [0.9627807486631016,
  0.9645989304812834,
  0.9639572192513369,
  0.9636363636363636,
  0.9628877005347594,
  0.9624598930481283,
  0.9629946524064171,
  0.9628877005347594,
  0.9623529411764706,
  0.9629946524064171])

## 선형분류

In [18]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [19]:
def classify_linear (any_df, rs):  # (df, random_state)
    df = load_to_df(any_df)
    df_c = df.copy()
    df_c['평균_폐업_비율'] = df_c['폐업_영업_개월_평균'] / df_c['서울_폐업_영업_개월_평균']
    df_c['평균_운영_비율'] = df_c['운영_영업_개월_평균'] / df_c['서울_운영_영업_개월_평균']
    X, y = df_c[['평균_폐업_비율', '평균_운영_비율']], df_c['상권_변화_지표']
    X_train, X_test, y_train, y_test = \
       train_test_split(X, y, stratify=y, random_state=rs)
    
    for C in [0.01, 1, 100, 1000, 10000]:  # C: 매개변수(크면 훈련 세트에 최대한 맞추고 작으면 가중치 계수 w가 0에 가까워지도록 조정)
        lr = LogisticRegression(C=C, max_iter=10000)
        lr.fit(X_train, y_train)
        print( C, ' : ', lr.score(X_train, y_train), lr.score(X_test, y_test))

In [20]:
classify_linear('상권-상권변화지표', 66)

0.01  :  0.7299108734402852 0.7264171122994653
1  :  0.9597504456327985 0.9593582887700535
100  :  0.9642067736185383 0.9614973262032086
1000  :  0.9642067736185383 0.9614973262032086
10000  :  0.9642067736185383 0.9614973262032086


## 나이브 베이즈 분류

In [21]:
from sklearn.naive_bayes import MultinomialNB

In [22]:
def classify_nb (any_df, rs):  # (df, random_state)
    df = load_to_df(any_df)
    df_c = df.copy()
    df_c['평균_폐업_비율'] = df_c['폐업_영업_개월_평균'] / df_c['서울_폐업_영업_개월_평균']
    df_c['평균_운영_비율'] = df_c['운영_영업_개월_평균'] / df_c['서울_운영_영업_개월_평균']
    X, y = df_c[['평균_폐업_비율', '평균_운영_비율']], df_c['상권_변화_지표']
    X_train, X_test, y_train, y_test = \
       train_test_split(X, y, stratify=y, random_state=rs)
    nb = MultinomialNB()
    nb.fit(X_train, y_train)
     
    return nb.score(X_train, y_train), nb.score(X_test, y_test)

In [23]:
classify_nb('상권-상권변화지표', 42)

(0.332620320855615, 0.332620320855615)

## 의사결정트리

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

In [25]:
# (df, random_state1, random_state2, max_depth, min_samples_leaf, max_leaf_nodes, criterion)
def classify_dt (any_df, rs1, rs2, md, msl, mln, crt):  
    df = load_to_df(any_df)
    df_c = df.copy()
    df_c['평균_폐업_비율'] = df_c['폐업_영업_개월_평균'] / df_c['서울_폐업_영업_개월_평균']
    df_c['평균_운영_비율'] = df_c['운영_영업_개월_평균'] / df_c['서울_운영_영업_개월_평균']
    X, y = df_c[['평균_폐업_비율', '평균_운영_비율']], df_c['상권_변화_지표']
    X_train, X_test, y_train, y_test = \
       train_test_split(X, y, stratify=y, random_state=rs1)
    dtc = DecisionTreeClassifier(max_depth=md, min_samples_leaf=msl,\
                                 max_leaf_nodes=mln, criterion=crt, random_state=rs2)
    dtc.fit(X_train, y_train)
    
    return dtc.score(X_train, y_train), dtc.score(X_test, y_test)

In [26]:
classify_dt('상권-상권변화지표', 18, 0, 5, 10, 4, 'entropy')

(0.9639215686274509, 0.966524064171123)

## 랜덤포레스트

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [28]:
# (df, random_forest1, random_forest2, n_estimators, n_jobs)
# n_estimators: 생성할 tree의 개수
# n_jobs: 사용할 코어 개수(-1로 지정하면 컴퓨터의 모든 코어 사용)
def classify_rf (any_df, rs1, rs2, n_est, n_jb):  
    df = load_to_df(any_df)
    df_c = df.copy()
    df_c['평균_폐업_비율'] = df_c['폐업_영업_개월_평균'] / df_c['서울_폐업_영업_개월_평균']
    df_c['평균_운영_비율'] = df_c['운영_영업_개월_평균'] / df_c['서울_운영_영업_개월_평균']
    X, y = df_c[['평균_폐업_비율', '평균_운영_비율']], df_c['상권_변화_지표']
    X_train, X_test, y_train, y_test = \
       train_test_split(X, y, stratify=y, random_state=rs1)
    rfc = RandomForestClassifier(random_state=rs2, n_estimators=n_est, n_jobs=n_jb)
    rfc.fit(X_train, y_train)
    
    return rfc.score(X_train, y_train), rfc.score(X_test, y_test)

In [29]:
classify_rf('상권-상권변화지표', 42, 66, 1000, -1)

(0.9811408199643494, 0.9622459893048129)

## 그래디언트 부스팅

In [30]:
from sklearn.ensemble import GradientBoostingClassifier

In [31]:
#(df, random_state1, random_state2, learning_rate, max_depth)
# learning_rate: 이전 트리의 오차를 얼마나 강하게 보정할 것인지 설정
# max_depth: 가지치기 최대 깊이
def classify_gb (any_df, rs1, rs2, lr, md):  
    df = load_to_df(any_df)
    df_c = df.copy()
    df_c['평균_폐업_비율'] = df_c['폐업_영업_개월_평균'] / df_c['서울_폐업_영업_개월_평균']
    df_c['평균_운영_비율'] = df_c['운영_영업_개월_평균'] / df_c['서울_운영_영업_개월_평균']
    X, y = df_c[['평균_폐업_비율', '평균_운영_비율']], df_c['상권_변화_지표']
    X_train, X_test, y_train, y_test = \
       train_test_split(X, y, stratify=y, random_state=rs1)
    gbc = GradientBoostingClassifier(random_state=rs2,
                                 learning_rate=lr,
                                 max_depth=md)
    gbc.fit(X_train, y_train)
    
    return gbc.score(X_train, y_train), gbc.score(X_test, y_test)

In [32]:
classify_gb('상권-상권변화지표', 42, 66, 1, 4)

(0.9807843137254902, 0.9621390374331551)

## 커널 서포트 벡터 머신

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

In [54]:
# (df, random_state, gamma, C)
# kernel: 주어진 데이터를 고차원 특징 공간으로 사상해주는 것(rbf가 성능이 제일 좋음)
# gamma: 하나의 데이터 샘플이 영향력을 행사하는 거리를 결정.
# C: 데이터 샘플들이 다른 클래스에 놓이는 것을 허용하는 정도 결정

def classify_svm (any_df, rs, gm, c):  
    df = load_to_df(any_df)
    df_c = df.copy()
    df_c['평균_폐업_비율'] = df_c['폐업_영업_개월_평균'] / df_c['서울_폐업_영업_개월_평균']
    df_c['평균_운영_비율'] = df_c['운영_영업_개월_평균'] / df_c['서울_운영_영업_개월_평균']
    X, y = df_c[['평균_폐업_비율', '평균_운영_비율']], df_c['상권_변화_지표']
    X_train, X_test, y_train, y_test = \
       train_test_split(X, y, stratify=y, random_state=rs)
    svc = SVC(kernel='rbf', gamma=gm, C=c, probability=True)
    svc.fit(X_train, y_train)
    
    return svc.score(X_train, y_train), svc.score(X_test, y_test)

In [55]:
classify_svm('상권-상권변화지표', 42, 10, 100)

(0.9647058823529412, 0.9637433155080214)